In [5]:
# !pip install textract

In [7]:
# !pip install -U nltk

In [9]:
# !pip install pdfminer3

In [13]:
# !pip install mammoth

In [15]:
# !pip install locationtagger

In [ ]:
import pandas as pd
from google.colab import drive
from pdfminer3.layout import LAParams
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import TextConverter
import io
import os
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import en_core_web_sm
nlp = en_core_web_sm.load()
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import mammoth
import locationtagger
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk.corpus import wordnet
nltk.download('wordnet')
from sklearn.decomposition import TruncatedSVD

In [ ]:
def pdf_extractor(path):
    r_manager = PDFResourceManager()
    output = io.StringIO()
    converter = TextConverter(r_manager, output, laparams=LAParams())
    p_interpreter = PDFPageInterpreter(r_manager, converter)
    with open(path, 'rb') as file:
        for page in PDFPage.get_pages(file,caching=True,check_extractable=True):
            p_interpreter.process_page(page)
            text = output.getvalue()
    converter.close()
    output.close()
    return text

In [ ]:
def read_files(file_path):
    fileTXT = []
     # This for loop is for reading all the files in file_path mentioned in the function
    for filename in os.listdir(file_path):
     # If the document is in pdf format then this code will be executed
    if(filename.endswith(".pdf")):
        try:
            fileTXT.append(pdf_extractor(file_path+filename)) # Here the pdf_extractor function is used to extract pdf file
        except Exception:
            print('Error reading pdf file :' + filename)
     # If the document is in docx format then this code will be executed
    if(filename.endswith(".docx")):
        try:
            with open(file_path + filename, "rb") as docx_file:
                result = mammoth.extract_raw_text(docx_file)
                text = result.value
                fileTXT.append(text)
        except IOError:
            print('Error reading .docx file :')
     # If the given document is in doc format then this loop will be executed
    if(filename.endswith(".doc")):
        try:
            text = textract.process(file_path+filename).decode('utf-8')
            fileTXT.append(text)
        except Exception:
            print('Error reading .doc file :' + filename)
     # If the given file in txt format then this file will be executed
    if(filename.endswith(".txt")):
        try:
            myfile = open(file_path+filename, "rt")
            contents = myfile.read()
            fileTXT.append(contents)
        except Exception:
            print('Error reading .txt file :' + filename)
    return fileTXT

In [ ]:
def preprocessing(Txt):
    sw = stopwords.words('english')
    space_pattern = '\s+'
    special_letters = "[^a-zA-Z#]"
    p_txt = []
    for resume in Txt:
        text = re.sub(space_pattern, ' ', resume) # Removes extra spaces
        text = re.sub(special_letters, ' ', text) # Removes special characters
        text = re.sub(r'[^\w\s]','',text) # Removes punctuations
        text = text.split() # Splits the words in a text
        text = [word for word in text if word.isalpha()] # Keeps alphabetic word
        text = [w for w in text if w not in sw] # Removes stopwords
        text = [item.lower() for item in text] # Lowercases the words
        p_txt.append(" ".join(text)) # Joins all the words back
    return p_txt

In [ ]:
jdTxt = read_files("jd_path")
jds = preprocessing(jdTxt)
TXT = p_resumeTxt+jds
# Finding TF-IDF score of all the resumes and JDs.
tv = TfidfVectorizer(max_df=0.85,min_df=10,ngram_range=(1,3))
# Converting TF-IDF to a DataFrame
tfidf_wm = tv.fit_transform(TXT)
tfidf_tokens = tv.get_feature_names()
df_tfidfvect1 = pd.DataFrame(data = tfidf_wm.toarray(),columns = tfidf_tokens)
print("\nTD-IDF Vectorizer\n")
print(df_tfidfvect1[0:10])

In [12]:
dimrec = TruncatedSVD(n_components=30, n_iter=7, random_state=42)
transformed = dimrec.fit_transform(df_tfidfvect1)

In [ ]:
vl = transformed.tolist()
# Converting list to DataFrame
fr = pd.DataFrame(vl)

In [ ]:
similarity = cosine_similarity(df_tfidfvect1[0:len(resumeTxt)],df_tfidfvect1[len(resumeTxt):])

In [ ]:
abc = []
for i in range(1,len(jds)+1):
    abc.append(f"JD {i}")
# DataFrame of similarity score
Data=pd.DataFrame(similarity,columns=abc)

In [ ]:
t = pd.DataFrame({'Original Resume':resumeTxt})
dt = pd.concat([Data,t],axis=1)

In [ ]:
def number(text):
    
 
    pattern = re.compile(r'([+(]?\d+[)\-]?[ \t\r\f\v]*[(]?\d{2,}[()\-]?[ \t\r\f\v]*\d{2,}[()\-]?[ \t\r\f\v]*\d*[ \t\r\f\v]*\d*[ \t\r\f\v]*)')
     # findall finds the pattern defined in compile
    pt = pattern.findall(text)
     # sub replaces a pattern matching in the text
    pt = [re.sub(r'[,.]', '', ah) for ah in pt if len(re.sub(r'[()\-.,\s+]', '', ah))>9]
    pt = [re.sub(r'\D$', '', ah).strip() for ah in pt]
    pt = [ah for ah in pt if len(re.sub(r'\D','',ah)) <= 15]
    for ah in list(pt):        
     # split splits a text
        if len(ah.split('-')) > 3: continue
            for x in ah.split("-"):
                try:
     # isdigit checks whether the text is number or not
                   if x.strip()[-4:].isdigit():
            
                      if int(x.strip()[-4:]) in range(1900, 2100):
                          pt.remove(ah)
                except: pass
                    
    number = None
    number = list(set(pt))
    return number              

In [ ]:
def email_ID(text):
 # compile helps us to define a pattern for matching it in the text
    r = re.compile(r'[A-Za-z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+')
    return r.findall(str(text))

In [ ]:
def location(text):
    place_entity = locationtagger.find_locations(text=text)
    return place_entity.cities
dt['Location']=dt['Resume'].apply(lambda x: location(x))

In [ ]:
def CompanyName(text):
 # for tagging each entity with its it’s labels
    tokens = nlp(str(text))
    x=[]
    for ent in tokens.ents:
        if ent.label_ == 'ORG':
    return ent.text

In [ ]:
wordcloud = WordCloud(width = 800, height = 500,background_color ='white',min_font_size = 10).generate(resumeTxt[17])

In [ ]:
plt.figure(figsize = (20, 5), facecolor = None)
plt.imshow(wordcloud)

In [ ]:
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()